In convolution neural network the hidden layers are convolution+max pooling.
So i just tried increasing it

In [ ]:
import dicom
import os
import pandas as pd

data_dir='D:/stage1/'
patients=os.listdir(data_dir)
labels_df=pd.read_csv('D:/stage1_labels.csv',index_col=0)

In [ ]:
import tensorflow as tf
import numpy as np

IMG_SIZE_PX=50
SLICE_COUNT=20

n_classes=2

x=tf.placeholder('float')
y=tf.placeholder('float')

keep_rate=0.8
keep_prob=tf.placeholder(tf.float32)

def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    #                        size of window         movement of window as you slide about
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')

In [ ]:
def convolutional_neural_network(x):
    #1st layer weights 5 x 5 x 5 patches, 1 channel, 32 features to compute.
    weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,5,1,32])),
               #2nd layer weights 5 x 5 x 5 patches, 32 channels, 64 features to compute.
               'W_conv2':tf.Variable(tf.random_normal([5,5,5,32,64])),
               #3rd layer weights 5 x 5 x 5 patches, 64 channels,128 features to compute.
               'W_conv3':tf.Variable(tf.random_normal([5,5,5,64,128])),
              #4th layer weights 5 x 5 x 5 patches, 128 channels, 256 features to compute.
               'W_conv4':tf.Variable(tf.random_normal([5,5,5,128,256])),
               'W_fc':tf.Variable(tf.random_normal([2293760,1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}
              #1st layer biases
    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
              #2nd layer biases
               'b_conv2':tf.Variable(tf.random_normal([64])),
              #3rd layer biases
               'b_conv3':tf.Variable(tf.random_normal([128])),
              #4th layer biases
               'b_conv4':tf.Variable(tf.random_normal([256])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    #                            image X      image Y        image Z
    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])
    #1st hidden layer
    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)
    #2nd hidden layer
    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)
    #3rd hidden layer
    conv3 = tf.nn.relu(conv3d(conv2, weights['W_conv3']) + biases['b_conv3'])
    conv3 = maxpool3d(conv3)
    #4th hidden layer
    conv4 = tf.nn.relu(conv3d(conv3, weights['W_conv4']) + biases['b_conv4'])
    conv4 = maxpool3d(conv4)
    #conv5 = tf.nn.relu(conv3d(conv4, weights['W_conv5']) + biases['b_conv5'])
    #conv5 = maxpool3d(conv5)
    #conv6 = tf.nn.relu(conv3d(conv5, weights['W_conv6']) + biases['b_conv6'])
    #conv6 = maxpool3d(conv6)
    
    
                              #tf input value
    fc = tf.reshape(conv4,[-1, 2293760])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']

    return output